In [1]:
%pylab inline
import pandas as pd
import numpy as np
import seaborn as sns
import re
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.decomposition.online_lda import LatentDirichletAllocation
import string
from sklearn.pipeline import Pipeline

Populating the interactive namespace from numpy and matplotlib


In [4]:
user = pd.read_pickle('data/user')
business =pd.read_pickle('data/business')
review = pd.read_pickle('data/review')
# top_restaurant = pd.read_csv('updated/high_reviews_restaurant.csv')

In [5]:
review.tail(2)

,user_id,review_id,text,votes.cool,business_id,votes.funny,stars,date,type,votes.useful
2685064,nELVJlkX8T0mUAArSPSJxw,vwmqHxxmy9rEAwhbkLXmnQ,he stood in the face of a 25 star biz and brav...,4,DH2Ujt_hwcMBIz8VvCb0Lg,5,3,2016-04-30,review,4
2685065,maAimqEE4G483rtifPKlYg,DDmiTM_jMhshjYkXk5Sshg,2 pm monday afternoon out of sour cream ridicu...,0,DH2Ujt_hwcMBIz8VvCb0Lg,0,1,2016-07-11,review,0


In [6]:
len(review)

1630712

In [7]:
%timeit Counter(review.business_id)

10 loops, best of 3: 105 ms per loop


In [ ]:
# Counter(review['business_id'])
# review_count = Counter(review['business_id'])

In [ ]:
review_count

In [ ]:
print(1)

In [8]:
review_count = Counter(review['business_id']) # number of unique restaurants
print(len(review_count))
top_restaurant = [(k,v)for k,v in review_count.items() if v > 300]#number of restaurants that have more than 300 reviews
sorted_by_count = sorted(top_restaurant, key=lambda tup: tup[1])
sorted_by_count[-5:]

26629


[('2e2e7WgqU1BnpxmQL5jbfw', 4103),
 ('aGbjLWzcrnEx2ZmMCFm3EA', 4149),
 ('zt1TpTuJ6y9n551sw9TaEg', 4333),
 ('sIyHTizqAiGu12XMLX3N3g', 4531),
 ('4bEjOyTaDG24SY5TxsaUNQ', 5558)]

In [4]:
def categorize_reviews(business_id):
    business_index = restaurant_review['business_id'] == business_id # get the business index
    one_restaurant = restaurant_review[business_index]
    good = one_restaurant.ix[one_restaurant['stars'] > 3.5]
    bad = one_restaurant.ix[one_restaurant['stars'] <= 3.5]
    good = good.reset_index()
    bad = bad.reset_index()
    return good.text,bad.text

In [5]:
def lda_tm(x):
    tfidf = TfidfVectorizer(stop_words='english')
    x = tfidf.fit_transform(x)
    lda = LatentDirichletAllocation(n_topics=15,
                                max_iter=5,
                                learning_method='online',
                                learning_offset=50.)
    terms = tfidf.get_feature_names()
    lda.fit(x)
    C = lda.components_
    for topic,term in enumerate(C):
        top_indices = np.argsort(C)[topic][::-1][:10]
        term_ranking = [terms[i] for i in top_indices]
        print("Topic {}: {}".format(topic+1, ", ".join(term_ranking)))
    

In [6]:
def nmf_tm(x):
    tfidf = TfidfVectorizer(stop_words='english')
    x = tfidf.fit_transform(x)
    nmf = NMF(init="nndsvd",
            n_components=15,
            max_iter=200)
    terms = tfidf.get_feature_names()
    nmf.fit(x)
    W = nmf.fit_transform(x)
    H = nmf.components_
    for topic,term in enumerate(H):
        top_indices = np.argsort(H)[topic][::-1][:10]
        term_ranking = [terms[i] for i in top_indices]
        print("Topic {}: {}".format(topic+1, ", ".join(term_ranking)))
    

In [7]:
len(review)

3873430

In [8]:
top_restaurant.head(2)

,Unnamed: 0,business_id,review_count
0,0,FY_OXEVcewAQFZuu6fynLA,301
1,1,7WGBi29_vj_7RcPVxpLNHA,301


In [9]:
rating_matrix = np.zeros( shape )

TypeError: expected sequence object with len >= 0 or a single integer

In [ ]:
self.SVD_matrix = np.zeros(shape = (944, 1683), dtype = np.float32)
def modifyRatingsDataset(self): 
        ''' Assign rating values to a matrix (userid, movieid ) '''
        for index, row in self.ratings.iterrows():
            self.SVD_matrix[ int(row[0]) , int(row[1])] = float(row[2])
        return csc_matrix(self.SVD_matrix, dtype=np.float32),self.SVD_matrix